In [ ]:
import pandas as pd
import numpy as np

from google.colab import drive
drive.mount('/content/gdrive')
data = pd.read_csv('/content/gdrive/MyDrive/CSCI 544/HW1/amazon_reviews_us_Office_Products_v1_00.tsv', sep='\t', usecols=['star_rating','review_body'])

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


<ipython-input-33-5922a310fa94>:6: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/content/gdrive/MyDrive/CSCI 544/HW1/amazon_reviews_us_Office_Products_v1_00.tsv', sep='\t', usecols=['star_rating','review_body'])


In [ ]:
! pip install contractions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 12.2 MB/s eta 0:00:00


In [ ]:
import re
import contractions

data = data.dropna()
data['review_body'] = data['review_body'].apply(lambda x : str(x).lower())
data['star_rating'] = data['star_rating'].map({1:1,2:1,3:1,4:2,5:2,'1':1,'2':1,'3':1,'4':2,'5':2})
data['review_body'] = [re.sub(r'<.*?>',' ',str(x)) for x in data['review_body']]
data['review_body'] = [re.sub(r'http\S+','',str(x)) for x in data['review_body']]

data['review_body'] = data['review_body'].apply(lambda x : contractions.fix(x))
data['review_body'] = [re.sub(r'[^a-zA-Z.!?]',' ',str(x)) for x in data['review_body']]
data['review_body']  = data['review_body'].apply(lambda x: re.sub(r'\s+', ' ', x))

def add_space_before_first_punctuation(text):
    punctuation_pattern = r'([.,?!]+|\.{2,})'
    def add_space(match):
        return f' {match.group(1)}'
    modified_text = re.sub(punctuation_pattern, add_space, text)
    return modified_text

# Apply the function to the 'text' column
data['review_body'] = data['review_body'].apply(lambda x: add_space_before_first_punctuation(x))

In [ ]:
data.head()

,star_rating,review_body
0,2,great product .
1,2,what is to say about this commodity item excep...
2,2,have not used yet but i am sure i will like it .
3,1,although this was labeled as new the one i rec...
4,2,gorgeous colors and easy to use


In [ ]:
data['token_count'] = data['review_body'].str.split().str.len()
filter_data = data[data['token_count'] < 35]
filter_data.head(5)

In [ ]:
filter_data.head(10)

In [ ]:
pos_data = filter_data[filter_data['star_rating'] == 2]
pos_data = pos_data[pos_data['review_body'].str.len() > 5]
neg_data = filter_data[filter_data['star_rating'] == 1]
neg_data = neg_data[neg_data['review_body'].str.len() > 5]

In [ ]:
filter_data[filter_data['review_body'] == 'cartridge was over filled black smears on paper every time i tried to use it . refund was quick though .']

In [ ]:
len(neg_data)

In [ ]:
pos_data = pos_data.sample(frac=1, random_state=22)
pos_data = pos_data[:254988]
print((pos_data.loc[:,'token_count'].mean() + neg_data.loc[:,'token_count'].mean())/2)

18.44854071564152


In [ ]:
groups_size = [500, 2000]
# neg 500 2000 125552
test_pos = pos_data[:groups_size[0]]
dev_pos = pos_data[groups_size[0]:groups_size[0]+groups_size[1]]
train_pos = pos_data[groups_size[0]+groups_size[1]:]

print("{} {} {}".format(len(test_pos), len(dev_pos), len(train_pos)))

test_neg = neg_data[:groups_size[0]]
dev_neg = neg_data[groups_size[0]:groups_size[0]+groups_size[1]]
train_neg = neg_data[groups_size[0]+groups_size[1]:]

print("{} {} {}".format(len(test_neg), len(dev_neg), len(train_neg)))

500 2000 252488
500 2000 252488


In [ ]:
%cd /content/gdrive/MyDrive/CSCI 544/project/style-transformer/data/amazon35

/content/gdrive/MyDrive/CSCI 544/project/style-transformer/data/amazon35


In [ ]:
# Extract the 'review_body' column as a list of strings

review_body_values = train_neg['review_body'].tolist()

# Open the binary file in binary write mode and write the text data
with open("train.neg", 'wb') as binary_file:
    for review_body in review_body_values:
      if len(review_body) > 5:
        binary_file.write(review_body.encode('utf-8')+ b'\n')

# **Evaluator acc.bin**

In [ ]:
pos = []
neg = []

with open('/content/gdrive/MyDrive/CSCI 544/project/style-transformer/data/amazon35/train.pos') as file :
  for line in file :
    pos.append(line)

with open('/content/gdrive/MyDrive/CSCI 544/project/style-transformer/data/amazon35/train.neg') as file :
  for line in file :
    neg.append(line)

In [ ]:
train = []
with open('/content/gdrive/MyDrive/CSCI 544/project/style-transformer/data/amazon35/train.txt', 'wb') as file :
  for line in pos :
    data = "__label__positive " + line
    file.write(data.encode('utf-8')+ b'\n')
    train.append(data)
  for line in neg :
    data = "__label__negative " + line
    file.write(data.encode('utf-8')+ b'\n')
    train.append(data)

In [ ]:
! pip install Cython==0.29.36
! pip install fasttext==0.8.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 17.7 MB/s eta 0:00:00
  Attempting uninstall: Cython
    Found existing installation: Cython 3.0.5
    Uninstalling Cython-3.0.5:
      Successfully uninstalled Cython-3.0.5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.0/74.0 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fasttext: filename=fasttext-0.8.3-cp310-cp310-linux_x86_64.whl size=1481035 sha256=ad0bb5c8f0138325fb7362dd52c62c0a29d46a7337d1ed189b8a7829a00fa5bf
  Stored in directory: /root/.cache/pip/wheels/2b/85/8b/3037ff2b0c1297bd87e6deee5c9a06e4b4da5f4c448c66f5f5
Successfully built fasttext


In [ ]:
import fasttext
classifier = fasttext.supervised('/content/gdrive/MyDrive/CSCI 544/project/style-transformer/data/amazon35/train.txt', "/content/gdrive/MyDrive/CSCI 544/project/style-transformer/evaluator/acc_amazon35", label_prefix='__label__')

In [ ]:
test_pos = []
test_neg = []

with open('/content/gdrive/MyDrive/CSCI 544/project/style-transformer/data/amazon35/test.pos') as file :
  for line in file :
    test_pos.append(line)

with open('/content/gdrive/MyDrive/CSCI 544/project/style-transformer/data/amazon35/test.neg') as file :
  for line in file :
    test_neg.append(line)

test = []
with open('/content/gdrive/MyDrive/CSCI 544/project/style-transformer/data/amazon35/test.txt', 'wb') as file :
  for line in test_pos :
    data = "__label__positive " + line
    file.write(data.encode('utf-8')+ b'\n')

  for line in test_neg :
    data = "__label__negative " + line
    file.write(data.encode('utf-8')+ b'\n')

In [ ]:
model = fasttext.load_model('/content/gdrive/MyDrive/CSCI 544/project/style-transformer/evaluator/acc_amazon35.bin', encoding='utf-8')
result = model.test('/content/gdrive/MyDrive/CSCI 544/HW1/amazon/data/test.txt')

In [ ]:
result = classifier.test('/content/gdrive/MyDrive/CSCI 544/project/style-transformer/data/amazon35/test.txt')
result.precision

0.892